## Leer todos los datos requeridos

In [0]:
%run "../includes/configuration"

In [0]:
movie_df = spark.read.parquet(f"{silver_folder_path}/movies")

In [0]:
genres_df = spark.read.parquet(f"{silver_folder_path}/genres")

In [0]:
movies_genres_df = spark.read.parquet(f"{silver_folder_path}/movies_genres")

### Join "genres" y "movie genres"

In [0]:
genres_mov_gen_df = genres_df.join(movies_genres_df, on='genre_id', how='inner') \
    .select('genre_name', 'movie_id')

## Join "movies_df" y "genres_mov_gen_df"

- Filtrar peliculas donde fecha de lanzamiento sea mayor o igual a 2015

In [0]:
movies_filtered_df= movie_df.filter('year_release_date >= 2015')

In [0]:
results_movies_genres_df = movies_filtered_df.join(genres_mov_gen_df, on='movie_id', how='inner')

In [0]:
results_df = results_movies_genres_df.select("year_release_date", "genre_name", "budget", "revenue")

In [0]:
from pyspark.sql.functions import sum, dense_rank, desc, asc
from pyspark.sql.window import Window

In [0]:
results_groupby_df = results_df.groupBy("year_release_date", "genre_name") \
.agg(sum("budget").alias("total_budget"), 
     sum("revenue").alias("total_revenue"))

In [0]:
results_dense_rank_df = Window.partitionBy("year_release_date").orderBy(desc("total_budget"), desc("total_revenue"))
final_df = results_groupby_df.withColumn("rank", dense_rank().over(results_dense_rank_df))

## Escribir datos en el datalake en formato Parquet

In [0]:
# final_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_group_movie_genre")
final_df.write.mode("overwrite").format('parquet').saveAsTable("movie_gold.results_group_movie_genre")

In [0]:
display(spark.read.parquet(f"{gold_folder_path}/results_group_movie_genre"))